In [7]:
import sys
sys.path.insert(0, "..")
from co2data.categories.concrete_category_provider import ConcreteCategoryProvider
from co2data.categories.importer import import_csv

csv_file = "../data/categories/test_categories.csv"
with open(csv_file, "r") as fh:
    category_list = import_csv(fh)
category_provider = ConcreteCategoryProvider(category_list)
#print(category_provider.get_all())

In [8]:
from co2data.pdf.directory_importer import import_bills_in_directory
from pathlib import Path
from co2data.bills.concrete_bill_store import ConcreteBillStore
from co2data.pdf.file_system_pdf_store import FileSystemPdfStore

bills_directory = Path("../pdf/")
pdf_store = FileSystemPdfStore()
bill_store = ConcreteBillStore(pdf_store)
import_bills_in_directory(bill_store, bills_directory)


In [9]:
from co2data.workflow.status_tracker import Status, StatusTracker

bill_status = {billid: Status.TO_DO for billid in bill_store.all()}
status_tracker = StatusTracker(bill_status)

In [11]:
from co2data.bills.bill import Position
from co2data.bills.bill_id import BillId
from ipywidgets import Layout
import ipywidgets as widgets
from wand.image import Image as WImage, Image

select_rows = 20
select_todo = widgets.Select(
    options=status_tracker.get_by_status(Status.TO_DO),
    rows=select_rows,
    description='ToDo:',
    disabled=False
)
select_inprogress = widgets.Select(
    options=status_tracker.get_by_status(Status.IN_PROGRESS),
    rows=select_rows,
    description='In Progress:',
    disabled=False
)
select_done = widgets.Select(
    options=status_tracker.get_by_status(Status.DONE),
    rows=select_rows,
    description='Done:',
    disabled=False
)

select_category = widgets.Dropdown(
    options=[category.name for category in category_provider.get_all()],
    description="Kategorie"
)

select_position = widgets.Select(
    description="Positionen"
)

textinput_position_value = widgets.Text(
    placeholder='Gebe den Wert ein',
    description='Wert:',
    disabled=False
)

unit_label = widgets.Label(
    value="test"
)

button_add_position = widgets.Button(
    description="Add"
)

button_delete_position = widgets.Button(
    description="Delete"
)

button_done = widgets.Button(
    description="Done"
)

log_output = widgets.Output()

image = widgets.Image(width=700, height=1000)

class CurrentBill:

    def __init__(self) -> None:
        self.bill_id = None

    @property
    def id(self) -> BillId | None:
        return self.bill_id

    def set_id(self, bill_id: BillId) -> None:
        self.bill_id = bill_id
        log(f"current bill: {self.bill_id}")
        show_pdf(self.bill_id)

current_bill = CurrentBill()


def handle_new_current_bill(event):
    current_bill.set_id(event.new)

def handle_new_category(event):
    category = category_provider.get_by_name(event.new)
    unit_label.value = category.unit

def log(message: str) -> None:
    with log_output:
        print(message)

def show_pdf(bill_id):
    filestream = pdf_store.get(bill_store.get(bill_id).file_identifier)
    img = Image(file=filestream, format="pdf", resolution=150)
    image.value = img.make_blob(format="png")
    image.format = "png"
    return img

def add_position(button: widgets.Button) -> None:
    bill = bill_store.get(current_bill.id)
    category = category_provider.get_by_name(select_category.value)
    bill.add_position(category, textinput_position_value.value)
    status_tracker.set_status(bill.id, Status.IN_PROGRESS)
    update_positions()
    update_status_lists()

def delete_position(button: widgets.Button) -> None:
    position_index = select_position.index
    bill = bill_store.get(current_bill.id)
    bill.delete_position(position_index)
    if len(bill.positions) == 0:
        status_tracker.set_status(bill.id, Status.TO_DO)
    update_positions()
    update_status_lists()

def handle_bill_done(button: widgets.Button) -> None:
    log(f"Done Button clicked for {current_bill.id}")
    status_tracker.set_status(current_bill.id, Status.DONE)
    update_status_lists()

def update_status_lists():
    select_todo.options = status_tracker.get_by_status(Status.TO_DO)
    select_inprogress.options = status_tracker.get_by_status(Status.IN_PROGRESS)
    select_done.options = status_tracker.get_by_status(Status.DONE)

def get_position_description(position: Position) -> str:
    return f"{position.category.name} {position.category.unit}: {position.amount}"

def update_positions():
    position_descriptions = [get_position_description(p) for p in bill_store.get(current_bill.id).positions]
    select_position.options = position_descriptions

# Layout
left_box = widgets.VBox([select_todo, select_inprogress, select_done])
middle_box = widgets.VBox([image])
lower_right_box = widgets.HBox([button_done], layout=Layout())
position_button_box = widgets.HBox([button_add_position, button_delete_position])
position_value_input_box = widgets.HBox([textinput_position_value, unit_label])
right_box = widgets.VBox([select_position, select_category, position_value_input_box, position_button_box, lower_right_box, log_output])
main_box = widgets.HBox([left_box, middle_box, right_box])
# Event bindings
select_todo.observe(handle_new_current_bill, names="value")
select_inprogress.observe(handle_new_current_bill, names="value")
select_done.observe(handle_new_current_bill, names="value")
select_category.observe(handle_new_category, names="value")
button_add_position.on_click(add_position)
button_delete_position.on_click(delete_position)
button_done.on_click(handle_bill_done)
# Display main container
display(main_box)
log("ready to go")
current_bill.set_id(status_tracker.get_by_status(Status.TO_DO)[0])